In [40]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
%autoreload
import sys
sys.path.insert(0, '../')

In [42]:
from src.utilities.mluar_utils import *

In [43]:
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset, Dataset, load_from_disk
import numpy as np
from einops import rearrange, reduce, repeat
import torch
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import pyplot as plt
import math
import pandas as pd
import pickle as pkl

pd.set_option('display.max_colwidth', None)


In [5]:
MULTI_LUAR_PATH =  "/mnt/swordfish-pool2/milad/multi-luar-reddit-model/"
LUAR_PATH =  "/mnt/swordfish-pool2/nikhil/LUAR/pretrained_weights/LUAR-MUD/"

In [6]:
# Load models
multiluar_model = AutoModel.from_pretrained(MULTI_LUAR_PATH, trust_remote_code=True)
luar_model = AutoModel.from_pretrained(LUAR_PATH, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("rrivera1849/LUAR-MUD")

In [59]:
# Load data
data_path = '/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_perGenre-HRS2.1/TA2/hrs_06-27-24_english_perGenre-HRS2.1/data/hrs_06-27-24_english_perGenre-HRS2.1_TA2_input'
ground_truth_path = '/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_perGenre-HRS2.1/TA2/hrs_06-27-24_english_perGenre-HRS2.1/groundtruth/hrs_06-27-24_english_perGenre-HRS2.1_TA2'
hiatus_data, _, _ = load_aa_data(data_path, ground_truth_path)

Loading:  /mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_perGenre-HRS2.1/TA2/hrs_06-27-24_english_perGenre-HRS2.1/data/hrs_06-27-24_english_perGenre-HRS2.1_TA2_input


In [60]:
# keep authors with only more than one text
authors_with_multiple_texts = [x[0] for x in hiatus_data.authorID.value_counts().to_dict().items() if x[1] > 1]
hiatus_data = hiatus_data[hiatus_data.authorID.isin(authors_with_multiple_texts)]

In [61]:
hiatus_data = hiatus_data.sample(2000)

In [65]:
np.median([len(x.split()) for x in hiatus_data_texts])

np.float64(159.0)

In [66]:
hiatus_data_texts = hiatus_data.fullText.tolist()
labels = hiatus_data['authorID'].tolist()

# Compute embeddings
max_seq_length = 192
hiatus_data_embeddings, _= get_luar_embeddings(hiatus_data_texts, multiluar_model, tokenizer, max_length=max_seq_length, batch_size=1, is_multi_luar=True)

In [26]:
#np.max([len(x.split()) for x in hiatus_data.fullText.tolist()])

### Experiment Design

- For each layer, we find pairs of ground-truth texts written by the same author where the corresponding layer's embedding scored them high compared to other layers
- For each layer, we take a sample of these pairs of texts and prompt ChatGPT to find which linguistic level they are similar

#### Step 1:

In [67]:
# Compute Multi-luar similarity matrices at every layer and average of the layers
muti_luar_layers_sims = [compute_similarities(hiatus_data_embeddings, hiatus_data_embeddings, layer=i) for i in range(7)]
#muti_luar_layers_sims.append(compute_similarities(hiatus_data_embeddings, hiatus_data_embeddings, layer=None))
muti_luar_layers_sims = np.stack(muti_luar_layers_sims)

In [68]:
# Compute significant pairs to layers
layer_to_sig_pairs = {layer: extract_sig_pairs_for_layer(hiatus_data_texts, muti_luar_layers_sims, layer) for layer in range(7)}

In [69]:
layer_x_pairs = []
for layer in [0, 1, 2, 3, 4, 5, 6]:
    print(layer, len(layer_to_sig_pairs[layer]))
    sample_of_pairs = layer_to_sig_pairs[layer][:10]
    layer_x_pairs += [{'text-1': x[0], 'text-2': x[1], 'z-score': x[2], 'layer-sim': x[3], 'layer': layer} for x in sample_of_pairs]
layer_x_pairs_df = pd.DataFrame(layer_x_pairs)

0 555
1 32
2 17
3 275
4 50
5 8
6 360


In [70]:
layer_x_pairs_df.sample(n=10).head(n=10)

,text-1,text-2,z-score,layer-sim,layer
59,"One Cat is Getting Way Too Big!\n\nWe have four cats currently (we used to have five) and we put out food each day for all of them, along with water obviously. However, one of the cats gets in and gobbles up the food for all of them before the others have much of a chance.\n\nHow can we get that cat to control herself or share better? I'd hate to have to segregate them during feeding time and babysit them one by one to make sure the fat cat eats only what she is supposed to.\n\nDoes anyone have any suggestions? Also, if you know how to get this cat on a diet, those tips would be welcome as well. I don't think she's healthy at the size she's at. Thanks!","<PERSON>\n\nthis was my favorite movie when i was 14 so i rewatched it just to see how it held up + i was in need of a lighthearted movie bc i’ve watched a lot of heavy ones lately.\n\nas far as political correctness goes this movie is far from it (made in 1953 and portrays native americans terribly) so keep that in mind, but it’s seriously just pure serotonin and i HIGHLY recommend it if you just need a cute movie to lift your spirits.\n\ncan i also just point out that this movie is 🏳️‍🌈🏳️‍🌈 af?? so funny that 14 y/o gaybutididn’tknowityet me loved this movie sm.","[0.35336807, -1.7503309, -0.66188, -0.52688986, 0.1992927, 0.8703334, 1.5161055]","[0.61085254, 0.44736424, 0.53195286, 0.5424436, 0.5988786, 0.65102834, 0.7012143]",6
19,"The <PERSON> of Inisherin\n\nI had pretty neutral feelings towards this going in. I enjoyed 3 Billboards immensely, but was a bit underwhelmed for In Bruges, still enjoying it for the most part. I have to say I was pleasantly blown away with how much I got out of this.\n\nI would go as far to say I got more out of this than any other of his films, and is easily one of the best films of the year. At no point was I bored or felt disengaged. The psuedo-spiritual aspect of this in particular was the most fascinating, and all the performances, shots, and pacing was quite fantastic. The comedy itself may be too dark for some was hysterical for me and built up in an amazing blend of shock and hilarity.\n\nQuite a few times, I found myself getting lost in the dialogue at how it applied to my own life, feeling things similar to the characters, different applications of their mentalities, and reflecting on how I've spent my time and priorities. Anytime a film can do this for me or aid me in this I fall in love with it, and this puts it far and beyond a large portion of modern cinema. If I had to equate this film to a particular feel I would say this runs in the same vein as a lot of <PERSON> films. The small island atmosphere of an intense drama between a small cast, taking on existential questions that often pit them against each other. Obviously this was far more comedic than <PERSON>, but the core to me remains the same.\n\nI highly recommend seeing this as soon as possible I may revisit it soon myself.\n\nViewed at local Mariemont Theater","Room\n\nRoom is definitely one of those heartfelt late bloomers in the year that we all love. It went undetected by many (including me) and suddenly appeared as a Best Picture nomination. I decided to watch it and am quite glad that I did so.\n\nFirst of all, the two leads absolutely carry this film. They delivered stellar performances and easily elevated the effectiveness of this film. They had so much chemistry as mother and son. As another note, that scene with <PERSON> rolling out of the truck almost made me pass out. It was mostly due to the fact that I honestly cared about the fate of the main characters, and that was easily the most suspenseful scene of the film (was so well done too).\n\nThe one thing I think I can judge this film on plot wise is that I would've liked to have seen more content from the real world, as well as more on how <PERSON> got into 'Room'.\n\nIs Room going to win Best Picture? Probably not. Is <PERSON> going to get Best Actress? She very well 

In [71]:
layer_x_pairs_df.to_json('../data/layer_to_pairs_signficance.json')

#### Step 2:

In [46]:
from datadreamer import DataDreamer
from datadreamer.llms import HFTransformers, ParallelLLM, OpenAI
from datadreamer.steps import DataFromPrompt, ProcessWithPrompt,  HFHubDataSource, DataSource, zipped, concat
from functools import partial
from transformers import QuantoConfig
from datasets import concatenate_datasets, load_dataset
import json


model = OpenAI(model_name="gpt-3.5-turbo", api_key='sk-proj-zTbZNk16Ik1pZnqLn38ZT3BlbkFJImq3pd7widkr7RzsC771') #kathy's lab
# model = HFTransformers(
#                 "meta-llama/Meta-Llama-3-8B-Instruct",
#                 quantization_config=QuantoConfig(weights="int8"),
#                 device=0,
#                 device_map="cuda",
#             )
# model.config.pad_token_id = model.config.eos_token_id

def gen_from_iterable_dataset(iterable_ds):
    yield from iterable_ds

def evaluate_text_similarities(data_path, document_pairs, linguistic_lvl_name, linguistic_lvl_desc):
    instruction = "Given the two Documents below, rate their <linguistic_lvl> on a scale from 1 to 5: Score 1 equals very low similarity and score 5 equals high similarity. First, give reasons for your score and then output the score. The output should be in the following format: {\"reasons\": \"explain your rating\",  \"score\": \"<json integer>\"}"
    instruction = instruction.replace("<linguistic_lvl>", linguistic_lvl_desc)
    
    with DataDreamer(data_path):
        datasource = DataSource('documents', Dataset.from_list(document_pairs))
        datasource = datasource.map(lambda row: {'inputs': 'Document 1:\n {} \n Document 2:\n {}'.format(row['text-1'], row['text-2'])}, auto_progress=False)
        ds_focus_questions = ProcessWithPrompt(
          "{} describe text similarity".format(linguistic_lvl_name),
          inputs={"inputs": datasource.output["inputs"]},
          args={
             "llm": model,
             "n": 1,
             "instruction": instruction
          },
          outputs={"generations": linguistic_lvl_name},
        ).select_columns([linguistic_lvl_name])
        
        zipped_step = zipped(datasource, ds_focus_questions)

        results_iter = zipped_step.output.dataset
        results_ds   = Dataset.from_generator(partial(gen_from_iterable_dataset, results_iter))

        return results_ds

In [44]:
layer_x_pairs_df = pd.read_json('../data/layer_to_pairs_signficance.json')

In [45]:
layer_x_pairs_df.layer.value_counts()

0    10
1    10
2    10
3    10
4    10
6    10
5     8
Name: layer, dtype: int64

In [47]:
# Given the two Documents below, rate their <linguistic_lvl> on a scale from 1 to 5: Score 1 equals very low similarity and score 5 equals high similarity. First, give reasons for your score and then output the score in the following output format: {\"reasons\": \"explain your rating\",  \"score\": \"<json integer>\"}
ling_phenomena = {
    'syntax': 'syntactic similarity',
    'semantic': 'semantic similarity',
    'lexical': 'lexical similarity',
    'discourse': 'discourse similarity'
}

In [48]:
layer_x_pairs = [row.to_dict() for idx, row in layer_x_pairs_df.iterrows()]

In [49]:
results = []
for key, val in ling_phenomena.items():
    results.append(evaluate_text_similarities('./output', layer_x_pairs, key, val))

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: ./output
[ 🤖 DataDreamer 💤 ] Step 'documents' was previously run and saved, but was outdated. 😞
[ 🤖 DataDreamer 💤 ] Step 'documents' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents' finished and is saved to disk. 🎉
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' was previously run and saved, but was outdated. 😞
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' finished and is saved to disk. 🎉
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity (select_columns)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity (select_columns)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'zipped(documents (map), syntax describe text s

In [50]:
all_results = concatenate_datasets(results)

In [51]:
all_results.save_to_disk('../data/described_similarities_ds')

Saving the dataset (0/1 shards):   0%|          | 0/272 [00:00<?, ? examples/s]

### Analyze layer to lingustic similarity:

In [52]:
all_results = load_from_disk('../data/described_similarities_ds')

In [53]:
all_results_df = all_results.to_pandas()

In [54]:
zscores= all_results_df['z-score'].tolist()

In [55]:
all_results_df[['text-1', 'text-2', 'layer', 'lexical', 'syntax', 'discourse', 'semantic']].head(n=50)

,text-1,text-2,layer,lexical,syntax,discourse,semantic
0,"<PERSON>\n\nI love this film. <PERSON>’s greatest achievement is undoubtedly this. I believe <PERSON> performance is the greatest ever put to film.\n—\nTogether, they embody a character that is just fascinating to watch. You never empathise with him, but you do not hate him. It always has stricken me that the <PERSON> is a product of his own environment. NYC is the villain, the enabler. The drugs, the gambling, the scum, the darkness, the violence, all just naturally exists around him and he is just a part of the city’s routine. He cannot break free of its immoral, corruptive nature. He is bound to it.\n—\n<PERSON> contrasts innocence with depravity. The lieutenant drops his kids to school and takes a hit straight after. He wakes up the next day next to a toddler, still wasted from the night before, the drug dealer shown trying to make ends meet for his family and the nun is corrupted. Despite being such a cynical film, I believe <PERSON> tries to shine glimpses of hope on his city.\n—\nThe film doesn’t really stop. It’s scene after scene of some misdeed, mistake or wrongdoing. It literally doesn’t stop gambling, drinking, destroying, or drugging. Neither does the lieutenant. Neither does New York. It’s a fast track to destruction in one way or another. There’s no escape. It never seems like his redemption is possible until the very end but by then it’s too late. The nightclubs, shootings, drugs, stack on top of one another, an evil momentum constantly building, and <PERSON>’s soaring use of music enhances this build up all the more. You feel doomed from the very start.\n—\nFerrara is fascinating in so many ways. This is his peak shit, and one of my all time favourite movies. It genuinely blows me away every time I see it. Underrated.","<PERSON>\n\n“I’m tired of feeling disgusted with myself.”\n\n“Maybe you’re supposed to feel disgusted with yourself when you’re a teenager.”\n\nit’s likeable, but it leaves me wanting more. i think it was too short, and i wish there was more backstory for <PERSON> and <PERSON> (and <PERSON> and <PERSON> as well) to better understand the dynamic there. i’ll probably have to read the book to get that though. also, near the beginning when <PERSON> and <PERSON> were watching desert hearts together and then started making out i was so SHOCKED. where are my people who love desert hearts at.....",0,None,"**Reasons for Rating:**\n\nDocument 1 and Document 2 have different structures, tones, and content. Document 1 discusses a film and analyzes its themes and characters in a detailed and critical manner. On the other hand, Document 2 seems to be a personal reflection or review with a more casual and conversational tone, mentioning specific scenes and expressing personal reactions. The syntactic structures, vocabulary, and overall style of writing in the two documents are quite distinct, with Document 1 being more analytical and descriptive, while Document 2 is more informal and opinionated.\n\n**Score:**\n\n{""reasons"": ""The two documents have different structures, tones, and content, with Document 1 being more analytical and descriptive, and Document 2 being more informal and opinionated. The syntactic structures, vocabulary, and overall style of writing in the two documents are quite distinct."", ""score"": ""1""}",None,None
1,"<PERSON>\n\nI love this film. <PERSON>’s greatest achievement is undoubtedly this. I believe <PERSON> performance is the greatest ever put to film.\n—\nTogether, they embody a character that is just fascinating to watch. You never empathise with him, but you do not hate him. It always has stricken me that the <PERSON> is a product of his own environment. NYC is the villain, the enabler. The drugs, the gambling, the scum, the darkness, the violence, all just naturally exists around him and he is just a part of the city’s routine. He cannot break free of its immoral, corruptive nature. He is bound to it.\n—\n<PERSON> con

In [56]:
all_results_df.layer.value_counts()

0    40
1    40
2    40
3    40
4    40
6    40
5    32
Name: layer, dtype: int64

In [59]:
def aggregate_score(list_of_scores):
    scores = []
    for json_str in list_of_scores:
        if json_str == None:
            continue
        try:
            if "\n\n" in json_str:
                json_str = json_str.split("\n")[-1].strip()

            json_str = json_str.replace("Output:", "").strip()
                
                
            json_obj = json.loads(json_str)            
            scores.append(int(json_obj['score']))
        except:
            print(json_str)
            print('==============')
    return round(np.mean(scores), 2)

# Aggregating layer score
layer_scores_df = all_results_df.groupby('layer').aggregate({
    'lexical': aggregate_score,
    'syntax': aggregate_score,
    'discourse': aggregate_score,
    'semantic': aggregate_score,
}).reset_index()

**Score:** {"reasons": "The content, language, tone, and structure of the two documents are very dissimilar, making their syntactic similarity low.", "score": "1"}
**** {"reasons": "The two documents have very different topics, tones, and contexts, making them semantically dissimilar.", "score": "1"}


In [60]:
layer_scores_df.head(n=7)

,layer,lexical,syntax,discourse,semantic
0,0,1.5,1.60,1.2,1.20
1,1,1.8,2.10,2.1,2.10
2,2,1.6,1.60,1.4,1.70
3,3,1.3,1.22,1.3,1.33
4,4,2.2,2.00,2.1,2.30
5,5,1.5,1.25,1.0,1.12
6,6,1.0,1.10,1.1,1.00
